### Read CSV files

In [169]:
import csv

codons = []

with open('data/side-by-side.csv', newline='') as csvfile:
    csvreader = iter(csv.reader(csvfile, delimiter=','))
    # Skip first row
    next(csvreader)

    for abspos, virus, vaccine in csvreader:
        codons.append({
            'abspos': abspos,
            'virus': virus,
            'vaccine': vaccine
        })

print(len(codons))


1274


In [170]:
import csv

codon_table = []

with open('data/codon-table-grouped.csv', newline='') as csvfile:
    csvreader = iter(csv.reader(csvfile, delimiter=','))
    # Skip first row
    next(csvreader)

    for aminoacid, codon in csvreader:
        codon_table.append({
            'aminoacid': aminoacid,
            'codon': codon,
        })

print(len(codon_table))

64


### Categorize codon table by amount of 'good' chars.


In [171]:
def calculate_codon_score(codon):
    score = 0

    for char in codon:
        if char is 'G' or char is 'C':
            score += 1

    return score

In [172]:
grouped_codon_table = { 0: [], 1: [], 2: [], 3: [] }

for codon_row in codon_table:
    codon = codon_row['codon']
    count = calculate_codon_score(codon)

    grouped_codon_table[count].append(codon_row)

### Find a codon with the highest possible score for each aminoacid

In [173]:
aminoacids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 's', 'S', 'T', 'V', 'W', 'Y']
best_codons = {}

for aminoacid in aminoacids:
    for score in range(3, 0, -1):
        for codon_row in grouped_codon_table[score]:
            if codon_row['aminoacid'] is aminoacid:
                best_codons[aminoacid] = codon_row['codon']
                continue

print(best_codons)
print(f"Found all: {len(aminoacids) == len(best_codons)}")

{'A': 'GCT', 'C': 'TGT', 'D': 'GAT', 'E': 'GAA', 'F': 'TTC', 'G': 'GGT', 'H': 'CAT', 'I': 'ATC', 'K': 'AAG', 'L': 'TTG', 'M': 'ATG', 'N': 'AAC', 'P': 'CCT', 'Q': 'CAA', 'R': 'AGA', 's': 'TGA', 'S': 'TCT', 'T': 'ACT', 'V': 'GTT', 'W': 'TGG', 'Y': 'TAC'}
Found all: True


### Target

> It is known that a higher fraction of G and C characters improves the efficiency of an mRNA vaccine.



In [174]:
def find_codon_with_score(score, target_aminoacid):
    for codon_row in grouped_codon_table[score]:
        if codon_row['aminoacid'] is target_aminoacid:
            return codon_row

    return False

In [175]:
for i in range(len(codons)):
    virus_codon = codons[i]['virus']
    virus_codon_score = calculate_codon_score(virus_codon)
  
    # Find target aminoacid
    target_aminoacid = None
    for codon_row in codon_table:
        if codon_row['codon'] == virus_codon:
            target_aminoacid = codon_row['aminoacid']
            break

    # Find codon with the highest score
    for score in range(3, 0, -1):
        suggestion = None
        
        # If virus codon has the best score, use that one.
        if(virus_codon_score is score):
            codons[i]['suggestion'] = virus_codon
            break

        # Else find another codon with this score
        best_codon = find_codon_with_score(score, target_aminoacid)
        if best_codon:
            codons[i]['suggestion'] = best_codon['codon']
            break

### Calculate score

In [176]:
match_count = 0

for codon in codons:
    if codon['vaccine'] == codon['suggestion']:
        match_count += 1

percentage = 100 / len(codons) * match_count

f"Result: {percentage}%"


'Result: 62.71585557299843%'

### Print results

In [177]:
print("abspos, virus, suggestion, vaccine")

for codon in codons:
    print(f"{codon['abspos']}, {codon['virus']}, {codon['suggestion']}, {codon['vaccine']}")

abspos, virus, suggestion, vaccine
0, ATG, ATG, ATG
3, TTT, TTC, TTC
6, GTT, GTC, GTG
9, TTT, TTC, TTC
12, CTT, CTC, CTG
15, GTT, GTC, GTG
18, TTA, CTC, CTG
21, TTG, CTC, CTG
24, CCA, CCC, CCT
27, CTA, CTC, CTG
30, GTC, GTC, GTG
33, TCT, AGC, TCC
36, AGT, AGC, AGC
39, CAG, CAG, CAG
42, TGT, TGC, TGT
45, GTT, GTC, GTG
48, AAT, AAC, AAC
51, CTT, CTC, CTG
54, ACA, ACC, ACC
57, ACC, ACC, ACC
60, AGA, CGC, AGA
63, ACT, ACC, ACA
66, CAA, CAG, CAG
69, TTA, CTC, CTG
72, CCC, CCC, CCT
75, CCT, CCC, CCA
78, GCA, GCC, GCC
81, TAC, TAC, TAC
84, ACT, ACC, ACC
87, AAT, AAC, AAC
90, TCT, AGC, AGC
93, TTC, TTC, TTT
96, ACA, ACC, ACC
99, CGT, CGC, AGA
102, GGT, GGC, GGC
105, GTT, GTC, GTG
108, TAT, TAC, TAC
111, TAC, TAC, TAC
114, CCT, CCC, CCC
117, GAC, GAC, GAC
120, AAA, AAG, AAG
123, GTT, GTC, GTG
126, TTC, TTC, TTC
129, AGA, CGC, AGA
132, TCC, TCC, TCC
135, TCA, AGC, AGC
138, GTT, GTC, GTG
141, TTA, CTC, CTG
144, CAT, CAC, CAC
147, TCA, AGC, TCT
150, ACT, ACC, ACC
153, CAG, CAG, CAG
156, GAC, GAC, 